In [2]:
# open tiktokers.json and read the data and print length of the data
import json

with open('tiktokers.json') as f:
    data = json.load(f)

print(len(data))

129


In [12]:
# convert the data to csv format and write it to a file
# ammong the keys of the data, I need id, nickname, ttSeller, signature, uniqueId, followerCount, followingCount, friendCount, heartCount, videoCount, diggCount, profileUrl, and country
import csv
import requests

with open('tiktokers.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'nickname', 'ttSeller', 'signature', 'uniqueId', 'followerCount', 'followingCount', 'friendCount', 'heartCount', 'videoCount', 'diggCount', 'profileUrl', 'language'])
    for t in data:
        # using ollama api, determine which country the tiktokers are from
        nickname = t["nickname"].replace("\n", " ")
        bio = t["signature"].replace("\n", " ")
        uniqueId = t["uniqueId"]

        system_promt = f"You will be given the nickname, and bio of a TikToker. Your job is to look at it and find out what language the TikToker's nickname and bio are written. Return only the language, nothing else. For example, if they're written by English, return “English” only."
        system_message = {
            "role": "system",
            "content": system_promt,
        }

        user_message = {
            "role": "user",
            "content": f"nickname: {nickname}\nbio: {bio}",
        }

        system_message_2 = {
            "role": "user",
            "content": "What language is the nickname and bio written in? Remember, return only the language, nothing else.",
        }

        response = requests.post(
            "http://localhost:11434/api/chat",
            json={"model": "llama3",
                  "messages": [system_message, user_message, system_message_2],
                  "stream": False,
                  },
        )
        language = response.json()["message"]["content"]

        writer.writerow(
            [
                t["id"],
                t["nickname"].replace("\n", " "),
                t.get("ttSeller", False),
                t["signature"].replace("\n", " "),
                t["uniqueId"],
                t["followerCount"],
                t["followingCount"],
                t["friendCount"],
                t["heartCount"],
                t["videoCount"],
                t["diggCount"],
                f"tiktok.com/@{t['uniqueId']}",
                language,
            ]
        )